#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

albert		  predict.ipynb  trained_model	 trained_model3  train.json
jsontoList.ipynb  test.json	 trained_model2  train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [0]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('train.json')!=True:
  !wget -O train.json https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json
if fileExists('test.json')!=True:
  !wget -O test.json https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_test.json

In [8]:
!ls

albert		  predict.ipynb  trained_model	 trained_model3  train.json
jsontoList.ipynb  test.json	 trained_model2  train.ipynb


##解析json

In [9]:
!pip install ijson

     |████████████████████████████████| 102kB 2.2MB/s 


In [0]:
import ijson
# 讀取數據
def LoadData(filepath):
    context = []
    question = []
    answersText = []
    count=0
    contextId=''
    with open(filepath) as file:
        parser = ijson.parse(file)
        contextId=''
        context2=''
        question2=''
        for prefix, event, value in parser:
            # print('prefix',prefix)
            # print('event',event)
            # print('value',value)
            if prefix=="data.item.paragraphs.item.id":
              #print('contextId',value)          
              contextId=value
            if prefix=="data.item.paragraphs.item.context":
              #print('context',value)
              context2=value
            if prefix=="data.item.paragraphs.item.qas.item.question":
              question2=value
              #print('question',value)
              #print('contextId',contextId)              
            if prefix=="data.item.paragraphs.item.qas.item.answers.item.text":
              #print('context2',context2)
              #print('answersText',value)
              #print('contextId',contextId)
              context.append(context2)
              question.append(question2)
              answersText.append(value+"[SEP]")
            #print("---"*50)
            # count=count+1
            # if count==51:
            #   break;
    return context,question,answersText

In [0]:
context,question,answersText=LoadData('train.json')

In [18]:
print(len(context))

26936


In [23]:
for i in range(500,1500):
    if len(context[i]) <= 450 and len(question[i]) <= 42 and len(answersText[i]) <= 16:
        print(context[i])
        print(question[i])
        print(answersText[i])
        print('---'*20)

中國和朝鮮戰俘指控美軍拿戰俘進行化學毒氣和細菌原子武器試驗，後來爆發著名的「巨濟島事件」。聯合國軍聲稱中國和朝鮮軍官控制了部分戰俘營，製造暴動，聯合國軍對於戰俘的暴動，僅使用了催淚彈和震盪彈，而中方誇大為化學武器。邊震瑕稱美軍利用戰俘營中爆發痢疾的機會進行不同療法的醫學實驗，從1950年10月到1951年8月，僅在第4集中營裡死於細菌性痢疾的戰俘就達4000多人。還指控美軍為醫學實驗需要而在凍傷或感染截肢時過分切除肢體，並引用美醫學博士稱讚血管外科手術在戰爭期間發展的話來作為拿戰俘做醫學實驗的證據。環球網報導美國還把中國被俘人員當作動物一樣解剖來醫學試驗。有人看到活體解剖而被美軍挖去雙眼。中方記錄的美方罪行還包括威脅中國和北韓兩國紅十字會代表的人身安全，如用槍撞擊和毆打他們。最終國際紅十字會代表奈德林宣布解散聯合紅十字會小組協調組，終止聯合訪問。
聯合國軍對於戰俘的暴動中使用震盪彈和什麼被中國跨大成化學武器？
催淚彈[SEP]
------------------------------------------------------------
中國和朝鮮戰俘指控美軍拿戰俘進行化學毒氣和細菌原子武器試驗，後來爆發著名的「巨濟島事件」。聯合國軍聲稱中國和朝鮮軍官控制了部分戰俘營，製造暴動，聯合國軍對於戰俘的暴動，僅使用了催淚彈和震盪彈，而中方誇大為化學武器。邊震瑕稱美軍利用戰俘營中爆發痢疾的機會進行不同療法的醫學實驗，從1950年10月到1951年8月，僅在第4集中營裡死於細菌性痢疾的戰俘就達4000多人。還指控美軍為醫學實驗需要而在凍傷或感染截肢時過分切除肢體，並引用美醫學博士稱讚血管外科手術在戰爭期間發展的話來作為拿戰俘做醫學實驗的證據。環球網報導美國還把中國被俘人員當作動物一樣解剖來醫學試驗。有人看到活體解剖而被美軍挖去雙眼。中方記錄的美方罪行還包括威脅中國和北韓兩國紅十字會代表的人身安全，如用槍撞擊和毆打他們。最終國際紅十字會代表奈德林宣布解散聯合紅十字會小組協調組，終止聯合訪問。
邊震瑕認為在第4集中營裡4000多名戰俘是因什麼而死？
細菌性痢疾[SEP]
------------------------------------------------------------
中國和朝鮮戰俘指控美軍拿戰俘進行化學毒氣和細菌原子武器試驗，後來爆發著名的「巨濟